<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Iterative_Imputation_for_Missing_Values_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.loli.net/2020/06/05/8tuPZ9oOUMW3aIJ.png" width="400px" class='center' alt='Horse Colic Dataset'>

In [8]:
''' 
1. Kaggle dataset https://www.kaggle.com/uciml/horse-colic?select=horse.csv
2. Github numeric dataset (categoric features transferred to numbers) https://raw.githubusercontent.com/jbrownlee/Datasets/master/horse-colic.csv
3. The goal is to predict whether or not a horse can survive based upon past medical conditions.
'''
!curl -o data.csv https://raw.githubusercontent.com/jbrownlee/Datasets/master/horse-colic.csv
!ls


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25151  100 25151    0     0   314k      0 --:--:-- --:--:-- --:--:--  314k
data.csv  sample_data


In [0]:
import pandas as pd
import numpy as np

# Explore data
Size: 300 rows x 28 columns

Name of target column is `cp_data`, where `1` means the horse survived and `2` perished. 


In [108]:
cols = ["surgery","age","hospital_number","rectal_temp","pulse","respiratory_rate","temp_of_extremities","peripheral_pulse","mucous_membrane","capillary_refill_time","pain","peristalsis","abdominal_distention","nasogastric_tube","nasogastric_reflux","nasogastric_reflux_ph","rectal_exam_feces","abdomen","packed_cell_volume","total_protein","abdomo_appearance","abdomo_protein","outcome","surgical_lesion","lesion_1","lesion_2","lesion_3","cp_data"]
df = pd.read_csv('data.csv', names=cols, na_values='?')
df

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,5.0,4.0,4.0,NaN,NaN,NaN,3.0,5.0,45.0,8.4,NaN,NaN,2.0,2,11300,0,0,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,3.0,4.0,2.0,NaN,NaN,NaN,4.0,2.0,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,3.0,3.0,1.0,NaN,NaN,NaN,1.0,1.0,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,2.0,4.0,4.0,1.0,2.0,5.0,3.0,NaN,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,533886,NaN,120.0,70.0,4.0,NaN,4.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,5.0,55.0,65.0,NaN,NaN,3.0,2,3205,0,0,2
296,2.0,1,527702,37.2,72.0,24.0,3.0,2.0,4.0,2.0,4.0,3.0,3.0,3.0,1.0,NaN,4.0,4.0,44.0,NaN,3.0,3.3,3.0,1,2208,0,0,1
297,1.0,1,529386,37.5,72.0,30.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,2.0,1.0,NaN,3.0,5.0,60.0,6.8,NaN,NaN,2.0,1,3205,0,0,2
298,1.0,1,530612,36.5,100.0,24.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,1.0,NaN,4.0,4.0,50.0,6.0,3.0,3.4,1.0,1,2208,0,0,1


In [109]:
"""201 perished v.s. 99 survived, this dataset is well-balanced.
"""
df.cp_data.value_counts()

2    201
1     99
Name: cp_data, dtype: int64

In [122]:
'''
1. Lots of columns have missing values.
2. Luckily, we do not have missing value on the target column
'''
for c in df.columns:
	# count number of rows with missing values
	n_miss = df[c].isna().sum()
	perc = n_miss / df.shape[0] * 100
	print('{:<25} missing {:<3} ({:>3.2f}%)'.format(c, n_miss, perc))

surgery                   missing 1   (0.33%)
age                       missing 0   (0.00%)
hospital_number           missing 0   (0.00%)
rectal_temp               missing 60  (20.00%)
pulse                     missing 24  (8.00%)
respiratory_rate          missing 58  (19.33%)
temp_of_extremities       missing 56  (18.67%)
peripheral_pulse          missing 69  (23.00%)
mucous_membrane           missing 47  (15.67%)
capillary_refill_time     missing 32  (10.67%)
pain                      missing 55  (18.33%)
peristalsis               missing 44  (14.67%)
abdominal_distention      missing 56  (18.67%)
nasogastric_tube          missing 104 (34.67%)
nasogastric_reflux        missing 106 (35.33%)
nasogastric_reflux_ph     missing 247 (82.33%)
rectal_exam_feces         missing 102 (34.00%)
abdomen                   missing 118 (39.33%)
packed_cell_volume        missing 29  (9.67%)
total_protein             missing 33  (11.00%)
abdomo_appearance         missing 165 (55.00%)
abdomo_protein    

In [107]:
"""Try imputer to fill the missing values
1. Convert data types to number, as some are object
"""
df1 = df.copy()
for c in df1.columns:
    df1[c] = pd.to_numeric(df1[c])


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

labels = df2['cp_data']
df3 = df2.drop(columns=['cp_data'])
test3 = df3.copy()

labels = df1['cp_data']
df3 = df1.drop(columns=['cp_data'])
test3 = df3.copy()


model = RandomForestClassifier()
parameters = {'verbosity': 0,
					  'learning_rate': 0.05,
					  'sumsampling': 1,
					  'colsample_bytree': 1,  # number of columns used by each tree
					  'max_depth': 10,
					  'gamma': 1,
					  'n_estimators': 3000
					  }
model = xgboost.XGBClassifier()
imputer = IterativeImputer()
pipeline = Pipeline(steps=[('i', imputer), ('m', model)])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)

%time scores = cross_val_score(pipeline, df3, labels, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise', verbose=1)
scores
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
# o = C_rf(df3, test3, labels)
# o.pipeline(eager=0)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 179 ms, sys: 5.1 ms, total: 184 ms
Wall time: 4.25 s
Mean Accuracy: 0.831 (0.048)


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    4.3s finished


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   surgery                300 non-null    object
 1   age                    300 non-null    int64 
 2   rectal_temp            300 non-null    object
 3   pulse                  300 non-null    object
 4   respiratory_rate       300 non-null    object
 5   temp_of_extremities    300 non-null    object
 6   peripheral_pulse       300 non-null    object
 7   mucous_membrane        300 non-null    object
 8   capillary_refill_time  300 non-null    object
 9   pain                   300 non-null    object
 10  peristalsis            300 non-null    object
 11  abdominal_distention   300 non-null    object
 12  nasogastric_tube       300 non-null    object
 13  nasogastric_reflux     300 non-null    object
 14  nasogastric_reflux_ph  300 non-null    object
 15  rectal_exam_feces      